In [12]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

In [20]:
# parameters
customer_id = '90'
formatted_attribute = 'Post Consumer Recycled Content Percent'
customer_name='%wbmason%'
buckets = """Office Chairs"""

strategy_version_input=761
attribute_id_input=5002

In [21]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [22]:
bucketz = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [23]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r', encoding='utf-8') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# _______________________________________________________________________________________________

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

# def find_first_digit(s, non=False):
#     for i, x in enumerate(s):
#         if x.isdigit() ^ non:
#             return i
#     return -1

# def split_digits(s, case=False):
#     non = True
#     while s:
#         i = find_first_digit(s, non)
#         if i == 0:
#             non = not non
#         elif i == -1:
#             yield int(s) if s.isdigit() else s if case else s.lower()
#             s = ''
#         else:
#             x, s = s[:i], s[i:]
#             yield int(x) if x.isdigit() else x if case else x.lower()

# def natural_key(s, *args, **kwargs):
#     return tuple(split_digits(s, *args, **kwargs))

# _______________________________________________________________________________________________

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
    
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
        
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))


def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

In [24]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(bucketz)]
print(len(df))
df

1


product_id external_id  \
0   434510669  5637489587   

                                                                        product_name  \
0  Flash Furniture Student Desk with Open Front Metal Book Box, 24"W x 18"D, Natural   

                                                                                                                                                                                                                                                           long_desc  \
0  Student Desk with Open Front Metal Book Box, 24"W x 18"D, Natural•Most preferred student desk with open book box•Designed for plenty of leg room•Spacious 24"W x 18"D work surface•5/8" Thick High Pressure Laminate Top•Laminate top with lacquer edge treatment   

   customer_id attributes                         buckets  bucket_id values  
0           90       None  Desks/Workstations/Workbenches       7440   None

In [7]:
# del df['product_id']
# del df['customer_id']
# del df['attributes']
# del df['bucket_id']
# del df['values']

In [9]:
df[df['attributes'].astype(str)!='None']

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values]
Index: []

In [8]:
## Good 
# (?i)(\d+\W?\d*\/?\d*\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d+\.\d+\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d*\W?\d+?\/?\d*\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d+\W?\d*\/?\d*"?”?'{0,2}\WW(?!eigh)(?:idth)?\b)|(\d+\.\d+'{0,2}\W?W(?!eigh)(?:idth)?\b)
# Experimental
# (?i)(\d+(?:\W?\d*\/?\d*)?\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d+(?:\.\d+)?\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d*\W?\d+\/?\d*\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d+(?:\W?\d*\/?\d*)?"?”?'{0,2}\WW(?!eigh)(?:idth)?\b)|(\d+\.\d+'{0,2}\W?W(?!eigh)(?:idth)?\b)

# Width

In [9]:
# regex_pattern=r'''(?i)(\d+(?:\W?\d*\/?\d*)?\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d+(?:\.\d+)?\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d*\W?\d+\/?\d*\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d+(?:\W?\d*\/?\d*)?"?”?'{0,2}\WW(?!eigh)(?:idth)?\b)|(\d+\.\d+'{0,2}\W?W(?!eigh)(?:idth)?\b)|()'''
regex_pattern=r'''(?i)(?i)((?<!\.)\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:(?:")?(?:”)?(?:'{0,2})?(?:\Winc?h?)?(?:\Wft)?(?:\Wfoot)?)\WW(?!eigh)(?:idth)?\b)|()'''
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

regex_pattern=r'''(?i)((?<!\.)\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:(?:")?(?:”)?(?:'{0,2})?(?:\Winc?h?)?(?:\Wft)?(?:\Wfoot)?)\WW(?!eigh)(?:idth)?\b)|(?i)((?<!\.)\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:(?:")?(?:”)?(?:'{0,2})?(?:\Winc?h?)?(?:\Wft)?(?:\Wfoot)?)\W[lwhd](?!eigh)(?:idth)?\b)|((?<!x)\s?\W\d+.?\d*\/?\d*\W?(?:(?:I?n?c?h?"?)?(?:f?e?e?t?'?)?(?:f?o?o?t?'?)?)\W?x.?\d+\W?\d*\/?\d*\W?(?:(?:I?n?c?h?"?)?(?:f?e?e?t?'?)?(?:f?o?o?t?'?)?)\W?\s(?!x\W?\d+))|(\d+\W?\d*\/?\d*\W?I?n?c?h?"?\W?x\W?\d+\W?\d*\/?\d*\W?I?n?c?h?"?\Wx?\W?\d*\W?\d*\/?\d*\W?I?n?c?h?"?)|(\d+.?\d?\/?\d*\\\"\s?W(?!eigh)\b)|(\d+\.\d*\\\"\s?W(?!eigh)\b)|()'''
df['matches_all'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))


regex_pattern_blank=r'(\d+\W?\d*\/?\d*)|()'
df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_width=df[(df['matches'].astype(str)!='[]')&(df['new_matches'].astype(str)!='[]')]
del df_width['new_matches']

df_width['final_matches']=df_width['matches'].apply(lambda x: re.sub(r'''(?i)(\\?\"?\”?(?:inc?h?)?(?<!').Wi?d?t?h?)|(\\?''\W?Wi?d?t?h?)|(\\" W)|(\\ in)''',' in',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x))).apply(lambda x: re.sub(r'''(\\?(?:(?<!')'(?!'))?(?:f?t?)?(?:f?o?o?t?)?\W?Wi?d?t?h?)''',' ft',str(x))).apply(lambda x: re.sub(r'(?i)\"xa0W|xa0W|xa\S?\S?',' in',str(x))).apply(lambda x: re.sub(r'''\[|\]|'|\\|"''','',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_width['final_matchez']=df_width['final_matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

df_width['final_matchez']=df_width['final_matchez'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x)))#.apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x)))                                                                          
df_width['final_matchez']=df_width['final_matchez'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
df_width['final_matchez']=df_width['final_matchez'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
df_width['final_matchez']=df_width['final_matchez'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))
df_width['final_matchez']=df_width['final_matchez'].apply(lambda x: re.sub(r'"\.','"0.',str(x)))


regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|()'         
df_width['rounding'] = df_width['final_matchez'].apply(lambda x: re_extract(regex_pattern_rounding, x))
df_width[df_width['rounding'].astype(str)!='[]']

Empty DataFrame
Columns: [external_id, product_name, long_desc, buckets, matches, matches_all, final_matches, final_matchez, rounding]
Index: []

In [10]:
df_width[curation_col]=df_width['final_matchez']
matches_natural=df_width[['external_id',curation_col]]
print(len(matches_natural))
matches_natural[curation_col].explode().value_counts()

203


["50 in"]                                                                                             15
["4.25 in"]                                                                                            7
["8 in","10 in"]                                                                                       7
["60 in"]                                                                                              7
["13.4 in","15.7 in","20.5 in","22.8 in","24.8 in"]                                                    7
["16 in","20 in","24 in"]                                                                              6
["3 in"]                                                                                               6
["30.5 in","40 in","55 in"]                                                                            6
["14 in"]                                                                                              6
["56 in"]                                              

In [ ]:
# regex_pattern=r'''(?i)(\d+\W?\d*\/?\d*\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d+\.\d+\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d*\W?\d+?\/?\d*\\"?”?'{0,2}\W?W(?!eigh)(?:idth)?\b)|(\d+\W?\d*\/?\d*"?”?'{0,2}\WW(?!eigh)(?:idth)?\b)|(\d+\.\d+'{0,2}\W?W(?!eigh)(?:idth)?\b)|()'''
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

# regex_pattern_blank=r'(\d+\W?\d*\/?\d*)|()'
# df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))



# df_width=df[(df['matches'].astype(str)!='[]')&(df['new_matches'].astype(str)!='[]')]
# df_width['final_matches']=df_width['matches'].apply(lambda x: re.sub(r'''(?i)\\?''\W?W''',' in',str(x))).apply(lambda x: re.sub(r'''(?i)\\"?”?'{2}? W|"?”?'?'?W|\"xa0W|xa0W|"''',' in',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# df_width['final_matches']=df_width['final_matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

# df_width['final_matches']=df_width['final_matches'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x)))                                                                          
# df_width['final_matches']=df_width['final_matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# df_width['final_matches']=df_width['final_matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# df_width['final_matches']=df_width['final_matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))
# df_width['final_matches']=df_width['final_matches'].apply(lambda x: re.sub(r'"\.','"0.',str(x)))
                                                               
# df_width[curation_col]=df_width['final_matches']
# matches_natural=df_width[['external_id',curation_col]]
# print(len(matches_natural))
# df_width

# Find more values

In [12]:
# pat=r'(?i)(\d+.?\d?\/?\d*\\\"\s?W(?!eigh)\b)|(\d+\.\d*\\\"\s?W(?!eigh)\b)|()'
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(pat, x))
regex_pattern_length=r'(?i)(\d+\.\d+.?.?.?Le?n?g?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Le?n?g?t?h?\b)|()'         
df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'(?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
df['matches_height'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?(?!eigh)\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?(?!eigh)\b)|(\b(?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?Wi?d?t?h?(?!eigh)(?!\/)\b)|(\b(?<!\.)(?<!\/)\d+(?!\.)[\W]?[\W]?Wi?d?t?h?(?!eigh)(?!\/)\b)|()'
df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|()"
# df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d|()'
# df['new_matches'] = df['matches_length'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['blank_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df['blank_matches_long'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

# Diameter

In [13]:
df_diam=df[(df['new_matches'].astype(str)=='[]')&(df['matches_diameter'].astype(str)!='[]')]
df_diam['matches_height']=df_diam['matches_diameter'].apply(lambda x: natsorted(x))
df_diam['height_final']=df_diam['matches_height'].apply(lambda x: re.sub(r'''(?i)(\\\"\s?diameter)|(\\?\"?\”?(?:inc?h?)?(?<!').(diameter|dia))|(\\?('')\W?(diameter|dia))|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(\\?(?:(?<!')'(?!'))?(?:f?t?)?(?:f?o?o?t?)?\W?(diameter|dia))|(square\W?foot)|(\'\\xa.{0,2}diameter)''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x)))#.apply(lambda x: re.sub(r'''(\'.?xa.{0,3}in ft)''',' ft',str(x)))#.apply(lambda x: re.sub(r'(?i)\"xa0W|xa0W',' in',str(x)))#.apply(lambda x: re.sub(r'''(?i)\'\\xa in''',' ft',str(x)))
df_diam['height_final']=df_diam['height_final'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
df_diam['height_final']=df_diam['height_final'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'" ','"',str(x))).apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r'''(?i)(\\")''','"',str(x)))
df_diam['height_final']=df_diam['height_final'].apply(lambda x: re.sub(r'''(?i)(\["\[")''','["',str(x))).apply(lambda x: re.sub(r'''(?i)("]"])''','"]',str(x))).apply(lambda x: re.sub(r', "',',"',str(x)))
df_diam['height_final']=df_diam['height_final'].apply(lambda x: re.sub(r'''(?i)(\["\[\\")''','["',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
df_diam['height_final']=df_diam['height_final'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
df_diam['height_final']=df_diam['height_final'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('(?i)height|\\?"\W?H',' in',str(x))).apply(lambda x: re.sub('(?i)in.?in',' in',str(x)))

df_diam['height_final']=df_diam['height_final'].apply(lambda x: re.sub(r'\.625','63',str(x))).apply(lambda x: re.sub(r'\.188','19',str(x))).apply(lambda x: re.sub(r'12\: in','12 in',str(x))).apply(lambda x: re.sub(r'''\\\\"\\\\xa\s?in''',' in',str(x))).apply(lambda x: re.sub(r'''  ''',' ',str(x)))
regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|()'         
df_diam['rounding'] = df_diam['height_final'].apply(lambda x: re_extract(regex_pattern_rounding, x))
df_diam[df_diam['rounding'].astype(str)!='[]']

Empty DataFrame
Columns: [external_id, product_name, long_desc, buckets, matches, matches_all, new_matches, matches_length, matches_height, matches_width, matches_diameter, blank_matches, blank_matches_long, height_final, rounding]
Index: []

In [14]:
df_diam[curation_col]=df_diam['height_final']
matches_diam=df_diam[['external_id',curation_col]]
print(len(matches_diam))
matches_diam

138


external_id                                          Q:width
1        63191417                                        ["20 in"]
3        62927321                                        ["47 in"]
4        62927130                                        ["48 in"]
5        62926935                                        ["49 in"]
6        62926836                                        ["49 in"]
7        62925600                                        ["81 in"]
203      69722225                                ["21 in","42 in"]
372      69601367                                ["25 in","36 in"]
399      69572117                            ["6.25 in","8.25 in"]
578      68260965                                      ["3.88 in"]
719      65528051                                     ["13.75 in"]
721      65480854                                        ["15 in"]
796      63862164                                ["25 in","35 in"]
2145     69728509                                      ["3.75 in"]
2146     69728508                                       ["4.5 in"]
2415     66341536                ["34 in","48 in","54 in","64 in"]
2419     66106814                                ["30 in","45 in"]
2520     61777699                                ["24 in","30 in"]
2524     60906427                                      ["1.38 in"]
2684     65443460                                       ["3.3 in"]
2687     69720552                                       ["3.5 in"]
2696     69719847                                         ["3 in"]
2715     62926270                                        ["20 in"]
2771     65527450                                ["1.5 in","5 in"]
2801     62925310                                        ["29 in"]
2925     69719601                                      ["11.4 in"]
3064     65512821                                        ["15 in"]
3137     61776937                                        ["30 in"]
4576     76434140                                         ["6 in"]
4579     62848985                                      ["6.25 in"]
4879     66341499                ["24 in","28 in","34 in","46 in"]
4985     65644256                                      ["17.5 in"]
5258     65177303                                         ["9 in"]
6026     60906342                                      ["1.38 in"]
6044     66734741                                      ["11.8 in"]
6046     69592590                                        ["18 in"]
6145     65980828                                        ["21 in"]
6171     64775623                                     ["35.25 in"]
8078     69716111                     ["4.12 in","4.25 in","6 in"]
9519     61754638                                        ["17 in"]
9521     63196818                                      ["5.25 in"]
9529     62927628                                        ["56 in"]
9530     62927444                                        ["56 in"]
9531     62927093                                        ["40 in"]
9532     62925631                                        ["35 in"]
9914     66404521                                      ["11.8 in"]
9915     66227205                                      ["3.35 in"]
9929     66106746                                        ["60 in"]
9930     66106029                                        ["21 in"]
10052    62925402                                        ["36 in"]
10053    62924184                                        ["36 in"]
10302    43316427                                       ["3.1 in"]
10337    69722226                ["22 in","40 in","48 in","54 in"]
10593    65494059                                        ["20 in"]
11399    69719849                                       ["5.5 in"]
11526    66341543                ["34 in","48 in","54 in","64 in"]
11527    66106982                                        ["56 in"]
11717    62925419                                        ["34 in"]
13036    69723740                                    

# Extra

In [ ]:
df_more=df[(df['matches_all'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['matches_width'].astype(str)!='[]')]
print(len(df_more))
# df_more

In [ ]:
df_more['matches_more']=df_more['matches_width'].apply(lambda x: re.sub(r'''\\" W|".?W|''.?W|”.?W|˝.?W|”.?W|” w|″ W''',' in',str(x))).apply(lambda x: re.sub(r'''\\' W|'.?W|'.?W''',' ft',str(x))).apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_more['matches_more']=df_more['matches_more'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

In [ ]:
df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x))).apply(lambda x: re.sub('ft in','ft',str(x)))                                                                           
df_more['matches_more']=df_more['matches_more'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
df_more['matches_more']=df_more['matches_more'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

df_more[curation_col]=df_more['matches_more']
matchesmore=df_more[['external_id',curation_col]]
print(len(matchesmore))

In [ ]:
df_na=df[(df['matches_all'].astype(str)=='[]')&(df['matches_length'].astype(str)=='[]')&(df['matches_height'].astype(str)=='[]')&(df['matches_width'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]#&(df['new_matches'].astype(str)!='[]')]
df_na[curation_col]='n/a'
matchesnadf=df_na[['external_id',curation_col]]
print(len(matchesnadf))
matchesnadf

# Two Min Max (Min is width)

In [ ]:
pat_split=r'''(?i)((?<!x)\s?\W\d+.?\d*\/?\d*\W?(?:(?:I?n?c?h?"?)?(?:f?e?e?t?'?)?(?:f?o?o?t?'?)?)\W?x.?\d+\W?\d*\/?\d*\W?(?:(?:I?n?c?h?"?)?(?:f?e?e?t?'?)?(?:f?o?o?t?'?)?)\W?\s(?!x\W?\d+))|()'''
df['matches_split'] = df['product_name'].apply(lambda x: re_extract(pat_split, str(x)))
regex_pattern_blank=r'(?i)(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['matchez'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
split=df[(df['matches_split'].astype(str)!='[]')&(df['matches_length'].astype(str)=='[]')]
print(len(split))
# split.head()

In [ ]:
split['matches_split']=split['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
split['matches_split']=split['matches_split'].str.get(0).apply(lambda x: x.lower())
split[['length', 'width']]=split['matches_split'].str.split("x",expand=True,)

In [ ]:
split['length']=split['length'].apply(lambda x: re.sub(r"\[\'",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|8x/1x|4x4 b|26x31 g|8x/1",'',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"2 24 in",'24 in',str(x))).apply(lambda x: re.sub(r"33 (?!in)",'33 in',str(x))).apply(lambda x: re.sub(r" ",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"5 inch",'31.5 in',str(x))).apply(lambda x: re.sub(r"20x24','20",'20 in',str(x)))                        
split['width']=split['width'].apply(lambda x: re.sub(r"\'\]",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|None| |4-f",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"24 in','20x24 in",'24 in',str(x))).apply(lambda x: re.sub(r"27b",'27 in',str(x))).apply(lambda x: re.sub(r"24(?! )",'24 in',str(x)))
print(len(split['width'].explode().value_counts()))
# split['width'].explode().value_counts()

In [ ]:
split=split[split['length'].astype(str)!=""]
split=split[split['width'].astype(str)!=""]

split_inch=split
pat_split=r'''(?i)(in)|()'''
split_inch['inch'] = split_inch['width'].apply(lambda x: re_extract(pat_split, str(x)))
split_inch=split_inch[split_inch['inch'].astype(str)!='[]']
split_inch['length']=split_inch['length'].apply(lambda x:re.sub(r'\s?in|\s?inch|ch','',str(x)))
split_inch['width']=split_inch['width'].apply(lambda x:re.sub(r'\s?in|\s?inch|ch','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
split_inch['length']=split_inch['length'].astype(float)
split_inch['width']=split_inch['width'].astype(float)
split_inch['Max'] = split_inch[['length', 'width']].values.max(1)
split_inch['Min'] = split_inch[['length', 'width']].values.min(1)
split_inch['Min']=split_inch['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
print(len(split_inch))
split_inch[curation_col]=split_inch['Min']
split_inch[curation_col]=split_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))


split_ft=split
pat_split=r'''(?i)(ft)|()'''
split_ft['inch'] = split_ft['length'].apply(lambda x: re_extract(pat_split, str(x)))
pat_split=r'''(?i)(ft)|()'''
split_ft['ft'] = split_ft['width'].apply(lambda x: re_extract(pat_split, str(x)))
split_ft=split_ft[(split_ft['ft'].astype(str)!='[]')&(split_ft['inch'].astype(str)!='[]')]
split_ft['length']=split_ft['length'].apply(lambda x:re.sub(r'\W?ft','',str(x)))
split_ft['width']=split_ft['width'].apply(lambda x:re.sub(r'\W?ft','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
split_ft['length']=split_ft['length'].astype(float)
split_ft['width']=split_ft['width'].astype(float)
split_ft['Max'] = split_ft[['length', 'width']].values.max(1)
split_ft['Min'] = split_ft[['length', 'width']].values.min(1)
split_ft['Min']=split_ft['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
print(len(split_ft))

split_ft[curation_col]=split_ft['Min']
split_ft[curation_col]=split_ft[curation_col].apply(lambda x: str(f"['{x} ft']")).apply(lambda x: re.sub(r"'",'"',str(x)))

In [ ]:
msplit_inch=split_inch[['external_id',curation_col]]
msplit_ft=split_ft[['external_id',curation_col]]

In [ ]:
msplit_inch[curation_col].explode().value_counts()

# Three Min Max (split)

In [ ]:
pat_split=r'(?i)(\d+\W?\d*\/?\d*\W?I?n?c?h?"?\W?x\W?\d+\W?\d*\/?\d*\W?I?n?c?h?"?\Wx?\W?\d*\W?\d*\/?\d*\W?I?n?c?h?"?)|()'
df['matches_split_three'] = df['product_name'].apply(lambda x: re_extract(pat_split, str(x)))
regex_pattern_blank=r'(?i)(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['matchez'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
split=df[df['matches_split'].astype(str)!='[]']
print(len(split))
# split.head()

In [ ]:
split['matches_split']=split['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
split['matches_split']=split['matches_split'].str.get(0).apply(lambda x: x.lower())
split[['length', 'width']]=split['matches_split'].str.split(" x ",expand=True,)

In [ ]:
split['length']=split['length'].apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|8x/1x|4x4 b|26x31 g|8x/1",'',str(x))).apply(lambda x: re.sub(r"2 24 in",'24 in',str(x))).apply(lambda x: re.sub(r"33 (?!in)",'33 in',str(x))).apply(lambda x: re.sub(r" ",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"5 inch",'31.5 in',str(x))).apply(lambda x: re.sub(r"20x24','20",'20 in',str(x)))                        
split['width']=split['width'].apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|None| |4-f",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"24 in','20x24 in",'24 in',str(x))).apply(lambda x: re.sub(r"27b",'27 in',str(x))).apply(lambda x: re.sub(r"24(?! )",'24 in',str(x)))

In [ ]:
# split=split[split['length'].astype(str)!=""]
# split=split[split['width'].astype(str)!=""]
# split['length']=split['length'].apply(lambda x:re.sub(r' in','',str(x)))#.apply(lambda x:str(f'"{x}"'))
# split['width']=split['width'].apply(lambda x:re.sub(r' in','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
# split['length']=split['length'].astype(float)
# split['width']=split['width'].astype(float)

# split['Max'] = split[['length', 'width']].values.max(1)
# split['Min'] = split[['length', 'width']].values.min(1)
# split['Min']=split['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# split['Max']=split['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [ ]:
# split[curation_col]=split['Min']
# split[curation_col]=split[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))

In [ ]:
# msplit=split[['external_id',curation_col]]
# msplit[curation_col]=msplit[curation_col]
# print(len(msplit))
# msplit[curation_col].explode().value_counts()

In [ ]:
print(len(df[(df['matchez'].astype(str)=='[]')&(df['matches_all'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')&(df['matches'].astype(str)=='[]')]))
na_values=df[(df['matchez'].astype(str)=='[]')&(df['matches_all'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')&(df['matches'].astype(str)=='[]')]
na_values[curation_col]='n/a'
matches_na_values=na_values[['external_id',curation_col]]
matches_na_values

In [ ]:
# del na_values['customer_id']
# del na_values['product_id']
# del na_values['attributes']
# del na_values['buckets']
# del na_values['bucket_id']
# del na_values['values']

# print(len(na_values))
# na_values

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnadf))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(msplit_ft))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(msplit_inch))


def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matches_natural))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matches_na_values))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(matches_diam))

In [ ]:
stop   

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, df, matches_diam)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, df, matchesnadf)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, df, msplit_ft)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, df, msplit_inch)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, df, matches_natural)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, df, matches_na_values)